In [14]:
%matplotlib ipympl
import pandas as pd
from biopandas.pdb import PandasPdb
from prody import parsePDBHeader
from typing import Optional
import numpy as np
import time
import matplotlib.pyplot as plt

standard_residues = ['LYS', 'LEU', 'THR', 'TYR', 'PRO', 'GLU', 'ASP', 'ILE', 'ALA', 'PHE', 'ARG', 'VAL', 'GLN', 'GLY', 'SER', 'TRP', 'CYS', 'HIS', 'ASN', 'MET', 'SEC', 'PYL']

In [2]:
def f1(x):
    return x ** -1

In [3]:
def f1_5(x):
    return x ** -1.5

In [15]:
def f2(x):
    return x ** -2

In [99]:
def f2_5(x):
    return x ** -2.5

In [6]:
def f3(x):
    return x ** -3

In [ ]:
def preprocess(pdb_path, feature, include='none', exclude='none'):

    try:
        cutoff = len(pdb_path[::-1].split('/',1)[1][::-1]) + 1
    except IndexError:
        cutoff = 0

    atomic_df = PandasPdb().read_pdb(pdb_path)

    header = parsePDBHeader(pdb_path)

    atomic_df = atomic_df.get_model(1)
    
    if include != 'none':
        for ind, i in enumerate(include):
            if ind == 0:
                temp = atomic_df.df['ATOM'][feature].eq(i)
            else:
                temp = temp | atomic_df.df['ATOM'][feature].eq(i)

        atomic_df.df['ATOM'] = atomic_df.df['ATOM'][temp]

    elif exclude != 'none':
        for ind, i in enumerate(exclude):
            if ind == 0:
                temp = atomic_df.df['ATOM'][feature].ne(i)
            else:
                temp = ~(~temp | ~atomic_df.df['ATOM'][feature].ne(i))

        atomic_df.df['ATOM'] = atomic_df.df['ATOM'][temp]

    atomic_df.to_pdb('pdbs/preprocessed/' + pdb_path[cutoff:])

In [16]:
def preprocess_startswith(pdb_path, feature: str = 'atom_name', include: list = ['C', 'N', 'O', 'S'], redefine_chains: bool = False):
    try:
        cutoff = len(pdb_path[::-1].split('/',1)[1][::-1]) + 1
    except IndexError:
        cutoff = 0

    atomic_df = PandasPdb().read_pdb(pdb_path)

    atomic_df = atomic_df.get_model(1)

    atomic_df.df['ATOM']['segment_id'] = ''
    atomic_df.df['ATOM']['element_symbol'] = ''

    temp = atomic_df.df['ATOM'][feature].eq(include[0])

    # try:
    #     if atomic_df.df['ATOM'].iloc[0]['element_symbol'][0] != atomic_df.df['ATOM'].iloc[0]['atom_name'][0]:
    #         redefine_atoms=True
    #     else:
    #         redefine_atoms=False
    # except IndexError:
    #     redefine_atoms=True

    split_occupancy = []
    added_residues = []

    if redefine_chains:
        residue_counter = atomic_df.df['ATOM'].iloc[0]['residue_number']
        chain = 65
    else:
        residue_counter = 0

    for i, x in atomic_df.df['ATOM'].iterrows():
        if x['residue_name'] in standard_residues or x['residue_name'] in added_residues:
            if x[feature][0] in include and x['occupancy'] > 0.5:
            
                temp.loc[i] = True

                if x['residue_number'] >= residue_counter and redefine_chains:
                    atomic_df.df['ATOM'].loc[i, 'chain_id'] = chr(chain)
                    residue_counter = x['residue_number']
                elif redefine_chains:
                    chain += 1
                    atomic_df.df['ATOM'].loc[i, 'chain_id'] = chr(chain)
                    residue_counter = x['residue_number']

                # if redefine_atoms:
                #     atomic_df.df['ATOM'].loc[i, 'element_symbol'] = x['atom_name'][0]

            if x[feature][0] in include and x['occupancy'] == 0.5:
                if x['residue_number'] >= residue_counter and redefine_chains:
                    atomic_df.df['ATOM'].loc[i, 'chain_id'] = chr(chain)
                    residue_counter = x['residue_number']
                elif redefine_chains:
                    chain += 1
                    atomic_df.df['ATOM'].loc[i, 'chain_id'] = chr(chain)
                    residue_counter = x['residue_number']

                if x['chain_id'] + str(x['residue_number']) + x['atom_name'] not in split_occupancy:
                    temp.loc[i] = True
                    split_occupancy += [x['chain_id'] + str(x['residue_number']) + x['atom_name']]
        else:
            yn = input(f"Would you like to include residue {x['residue_name']}? y/n: ")
            if yn == 'y':
                added_residues += [x['residue_name']]
                if x[feature][0] in include and x['occupancy'] > 0.5:
                
                    temp.loc[i] = True

                    if x['residue_number'] >= residue_counter and redefine_chains:
                        atomic_df.df['ATOM'].loc[i, 'chain_id'] = chr(chain)
                        residue_counter = x['residue_number']
                    elif redefine_chains:
                        chain += 1
                        atomic_df.df['ATOM'].loc[i, 'chain_id'] = chr(chain)
                        residue_counter = x['residue_number']

                    # if redefine_atoms:
                    #     atomic_df.df['ATOM'].loc[i, 'element_symbol'] = x['atom_name'][0]

                if x[feature][0] in include and x['occupancy'] == 0.5:
                    if x['residue_number'] >= residue_counter and redefine_chains:
                        atomic_df.df['ATOM'].loc[i, 'chain_id'] = chr(chain)
                        residue_counter = x['residue_number']
                    elif redefine_chains:
                        chain += 1
                        atomic_df.df['ATOM'].loc[i, 'chain_id'] = chr(chain)
                        residue_counter = x['residue_number']

                    if x['chain_id'] + str(x['residue_number']) + x['atom_name'] not in split_occupancy:
                        temp.loc[i] = True
                        split_occupancy += [x['chain_id'] + str(x['residue_number']) + x['atom_name']]

    atomic_df.df['ATOM'] = atomic_df.df['ATOM'][temp]

    atomic_df.to_pdb('pdbs/preprocessed/' + pdb_path[cutoff:])

    return 'pdbs/preprocessed/' + pdb_path[cutoff:]

In [17]:
def calc_exposure(pdb_path, funcs: dict = {'2': f2}, assignment=None, plus_inv_score: bool = True, save_matrix: bool = False, save_scores_as_vector: bool = False):

    try:
        cutoff = len(pdb_path[::-1].split('/',1)[1][::-1]) + 1
    except IndexError:
        cutoff = 0

    atomic_df = PandasPdb().read_pdb(pdb_path)

    atomic_df = atomic_df.get_model(1)

    coords = np.vstack((atomic_df.df['ATOM']['x_coord'].to_numpy(), atomic_df.df['ATOM']['y_coord'].to_numpy(), atomic_df.df['ATOM']['z_coord'].to_numpy())).T

    pair_scores = {}

    out = []

    for key in funcs:
        pair_scores[key] = np.zeros((len(coords), len(coords)))

    for i, coord1 in enumerate(coords):
        for j, coord2 in enumerate(coords[i+1:]):
            distance = np.linalg.norm(coord1-coord2)
            for key, func in funcs.items():
                pair_scores[key][i,i+j+1] = pair_scores[key][i+j+1,i] = func(distance)

    if assignment == None:
        for key, mat in pair_scores.items():
            temp = np.sum(mat, axis = 0)
            atomic_df.df['ATOM']['b_factor'] = temp
            atomic_df.to_pdb('pdbs/out/' + pdb_path[cutoff:-4] + '_' + key + '.pdb')

            out += ['pdbs/out/' + pdb_path[cutoff:-4] + '_' + key + '.pdb']
            
            if save_scores_as_vector:
                np.save('pdbs/out/npys/' + pdb_path[cutoff:-4] + '_' + key + '_vec.npy', temp)

            if plus_inv_score:
                atomic_df.df['ATOM']['b_factor'] = 100/temp
                atomic_df.to_pdb('pdbs/out/' + pdb_path[cutoff:-4] + '_' + key + '_inv.pdb')
                out += ['pdbs/out/' + pdb_path[cutoff:-4] + '_' + key + '.pdb']
                if save_scores_as_vector:
                    np.save('pdbs/out/npys/' + pdb_path[cutoff:-4] + '_' + key + '_inv_vec.npy', 100/temp)

            if save_matrix:
                np.save('pdbs/out/npys/' + pdb_path[cutoff:-4] + '_' + key + '_mat.npy', mat)
    
    if type(assignment) == dict:
        mat_not_saved = True
        for k, assigment_vert in assignment.items():
            for key, mat in pair_scores.items():
                temp = assigment_vert @ mat
                atomic_df.df['ATOM']['b_factor'] = temp
                atomic_df.to_pdb('pdbs/out/' + pdb_path[cutoff:-4] + '_' + k + '_' + key + '.pdb')
                out += ['pdbs/out/' + pdb_path[cutoff:-4] + '_' + k + '_' + key + '.pdb']

                if save_scores_as_vector:
                    np.save('pdbs/out/npys/' + pdb_path[cutoff:-4] + '_' + k + '_' + key + '_vec.npy', temp)

                if plus_inv_score:
                    atomic_df.df['ATOM']['b_factor'] = 100/temp
                    atomic_df.to_pdb('pdbs/out/' + pdb_path[cutoff:-4] + '_' + k + '_' + key + '_inv.pdb')
                    out += ['pdbs/out/' + pdb_path[cutoff:-4] + '_' + k + '_' + key + '.pdb']
                    if save_scores_as_vector:
                        np.save('pdbs/out/npys/' + pdb_path[cutoff:-4] + '_' + k + '_' + key + '_inv_vec.npy', 100/temp)

                if save_matrix and mat_not_saved:
                    np.save('pdbs/out/npys/' + pdb_path[cutoff:-4] + '_' + key + '_mat.npy', mat)
                    mat_not_saved = False

    return out

In [8]:
def print_features(pdb_path, feature):
    atomic_df = PandasPdb().read_pdb(pdb_path)

    header = parsePDBHeader(pdb_path)

    atomic_df = atomic_df.get_model(1)

    out = []

    for chain in atomic_df.df['ATOM'][feature]:
        if chain not in out:
            out = out + [chain]

    print(out)
    # return out

In [11]:
def create_vectors(pdb_path, chains, feature):
    atomic_df = PandasPdb().read_pdb(pdb_path)

    header = parsePDBHeader(pdb_path)

    atomic_df = atomic_df.get_model(1)

    out = np.ones((len(chains), len(atomic_df.df['ATOM'])))

    for ind, chain_id in enumerate(atomic_df.df['ATOM'][feature]):
        for i, chain in enumerate(chains):
            if chain_id not in chain:
                out[i,ind] = 0

    return out

In [18]:
def create_3_vectors(pdb_path, chain1, feature):
    atomic_df = PandasPdb().read_pdb(pdb_path)
    atomic_df = atomic_df.get_model(1)

    out_tot = np.ones(len(atomic_df.df['ATOM']))

    if type(chain1) == str:
        out1 = np.array(atomic_df.df['ATOM'][feature].eq(chain1)).astype(int)
        name = chain1
    elif type(chain1) == list:
        for ind, chain in enumerate(chain1):
            if ind == 0:
                temp = atomic_df.df['ATOM'][feature].eq(chain)
                name = chain + 'plus'
            else:
                temp = temp | atomic_df.df['ATOM'][feature].eq(chain)
        out1 = np.array(temp).astype(int)

    out2 = out_tot-out1


    return {name: out1, 'not'+name: out2, 'tot': out_tot}

In [19]:
def average_score_per_residue(path):
    if path[-3:] == 'pdb':
        atomic_df = PandasPdb().read_pdb(path)

        atomic_df = atomic_df.get_model(1)

        current_residue = atomic_df.df['ATOM'].iloc[0].loc['chain_id'] + str(atomic_df.df['ATOM'].iloc[0].loc['residue_number'])

        scores = np.zeros(len(atomic_df.df['ATOM']))
        backbone_scores = np.zeros(len(atomic_df.df['ATOM']))

        atom_count = 0
        score = 0
        backbone_atom_count = 0
        backbone_score = 0

        for i, x in atomic_df.df['ATOM'].iterrows():
            if x['chain_id'] + str(x['residue_number']) == current_residue:
                atom_count+=1
                score+=x['b_factor']
                if x['atom_name'] in ['C', 'N', 'O', 'CA']:
                    backbone_atom_count+=1
                    backbone_score+=x['b_factor']
            else:
                for j in range(atom_count):
                    scores[i-j-1] = score / atom_count
                    if backbone_atom_count != 0:
                        backbone_scores[i-j-1] = backbone_score / backbone_atom_count

                atom_count=1
                score=x['b_factor']
                current_residue = x['chain_id'] + str(x['residue_number'])

                if x['atom_name'] in ['C', 'N', 'O', 'CA']:
                    backbone_atom_count=1
                    backbone_score=x['b_factor']
                else:
                    backbone_atom_count=0
                    backbone_score=0

        atomic_df.df['ATOM']['b_factor'] = scores
        atomic_df.to_pdb(path[:-4] + '_avgbyres.pdb')

        atomic_df.df['ATOM']['b_factor'] = backbone_scores
        atomic_df.to_pdb(path[:-4] + '_avgbyresbb.pdb')

    elif path[-7:] == 'defattr':
        localres = pd.read_csv(path, sep = '\t', header = 3, usecols = [1,2], names = ['atom', 'localres']).set_index('atom')

        current_residue = localres.iloc[0].name.split('@')[0]

        scores = np.zeros(len(localres))
        backbone_scores = np.zeros(len(localres))

        atom_count = 0
        score = 0
        backbone_atom_count = 0
        backbone_score = 0
        i=0

        for ind, res in localres.iterrows():
            if ind.split('@')[0] == current_residue:
                atom_count+=1
                score+=res['localres']
                if ind.split('@')[1] in ['C', 'N', 'O', 'CA']:
                    backbone_atom_count+=1
                    backbone_score+=res['localres']
            else:
                for j in range(atom_count):
                    scores[i-j-1] = score / atom_count
                    if backbone_atom_count != 0:
                        backbone_scores[i-j-1] = backbone_score / backbone_atom_count

                atom_count=1
                score=res['localres']
                current_residue = ind.split('@')[0]

                if ind.split('@')[1] in ['C', 'N', 'O', 'CA']:
                    backbone_atom_count=1
                    backbone_score=res['localres']
                else:
                    backbone_atom_count=0
                    backbone_score=0
            i+=1

        score_df = pd.read_csv(path, sep = '\t', header = 3, names = ['', 'atom', 'localres'])
        score_df['localres'] = scores
        score_df.to_csv(path[:-8] + '_avgbyres.defattr', sep = '\t', header=['attribute: locres \nrecipient: atoms \nmatch mode: 1-to-1', '', ''], index=False)

        score_df['localres'] = backbone_scores
        score_df.to_csv(path[:-8] + '_avgbyresbb.defattr', sep = '\t', header=['attribute: locres \nrecipient: atoms \nmatch mode: 1-to-1', '', ''], index=False)

In [20]:
def plot_score_v_localres_byatom(pdbout, defattr, backboneonly: bool = False, inverse: bool = True, interactive: bool = True):

    plt.close()
    
    fig,ax = plt.subplots()

    localres = pd.read_csv(defattr, sep = '\t', header = 3, usecols = [1,2], names = ['atom', 'localres']).set_index('atom')

    out = np.zeros((len(localres), 2))

    names = list(np.zeros(len(localres)).astype(int).astype(str))

    atomic_df = PandasPdb().read_pdb(pdbout)

    atomic_df = atomic_df.get_model(1)

    df = atomic_df.df['ATOM'].set_index(['chain_id','residue_number', 'atom_name'])

    i=0

    errorcount = 0

    if localres.iloc[0].name[0] == '#':
        k = len( localres.iloc[0].name.split('/')[0] )
    else:
        k = 0

    for ind, row in localres.iterrows():
        if not backboneonly or ind.split('@')[1] in ['C', 'N', 'O', 'CA']:
            try:
                if type(df.loc[(ind[1+k:2+k], int(ind[3+k:].split('@')[0])), 'b_factor'][ind[3+k:].split('@')[1]]) == pd.core.series.Series:
                    errorcount+=1
                else:
                    out[i,0] = df.loc[(ind[1+k:2+k], int(ind[3+k:].split('@')[0])), 'b_factor'][ind[3+k:].split('@')[1]]
                    out[i,1] = row['localres']
                    names[i] = ind
                    i+=1
            except KeyError:
                errorcount+=1
        else:
            errorcount+=1

    if errorcount != 0:
        out = out[:-errorcount].T
        names = names[:-errorcount]
    else:
        out=out.T

    if inverse:
        sc = plt.scatter(out[0], 1/out[1], s=3)#, color=(0,0,1,0.5))
        plt.plot(np.unique(out[0]), np.poly1d(np.polyfit(out[0], 1/out[1], 1))(np.unique(out[0])), color = (0,0,0))
        plt.ylabel('Local Resolution at Atom in Model (1/Å)')
    else:
        sc = plt.scatter(out[0], out[1], s=3)#, color=(0,0,1,0.5))
        plt.ylabel('Local Resolution at Atom in Model (Å)')
    plt.xlabel('Atom Exposure Score (Arbitrary Units)')

    if interactive:
        annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points", bbox=dict(boxstyle="round", fc="w"), arrowprops=dict(arrowstyle="->"))
        annot.set_visible(False)
        
        def update_annot(ind):
            pos = sc.get_offsets()[ind["ind"][0]]
            annot.xy = pos
            text = "{}".format(" ".join([names[n] for n in ind["ind"]]))
            annot.set_text(text)
            # annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
            annot.get_bbox_patch().set_alpha(0.4)
            
        def hover(event):
            vis = annot.get_visible()
            if event.inaxes == ax:
                cont, ind = sc.contains(event)
                if cont:
                    update_annot(ind)
                    annot.set_visible(True)
                    fig.canvas.draw_idle()
                else:
                    if vis:
                        annot.set_visible(False)
                        fig.canvas.draw_idle()

        fig.canvas.mpl_connect("motion_notify_event", hover)

    plt.show()

## Example process below:
(for soluble protein)

In [ ]:
initial_pdb_path = 'pdbs/3jcz.pdb'

preprocessed_pdb_path = preprocess_startswith(initial_pdb_path) # Preprocesses down to only C, N, O, S, and Se atoms, removes any low occupancy atoms, and removes segment_id and element_symbol (which often cause errors in ChimeraX)

calculated_pdb_paths = calc_exposure(preprocessed_pdb_path) # By default, calculates with d^-2 and gives score and inverse score (better for ChimeraX visualisation)

for pdbpath in calculated_pdb_paths: # This averages for every file generated by calc_exposure. You can choose any or all that you want explicitly instead
    average_score_per_residue(pdbpath) # By default, assigns each atom in a residue the average score of that atoms in that residue, and separately assigns each atom in a residue the average score of the backbone atoms in that residue

## Example process below:
(for membrane protein, calculating scores separately for protein exposure, membrane exposure, and total exposure)

In [ ]:
initial_pdb_path = 'pdbs/md_raw/amtb_md.pdb'

preprocessed_pdb_path = preprocess_startswith(initial_pdb_path) # Preprocesses down to only C, N, O, S, and Se atoms, removes any low occupancy atoms, and removes segment_id and element_symbol (which often cause errors in ChimeraX)
                                                                # This will also ask if you want to include nonstandard residues, such as HSD (protonated histidine) and BDD (in this case, detergent).

print_features(preprocessed_pdb_path, 'residue_name') # You can select any feature, another useful options can be 'chain_id' if chains are assigned correctly. If you already know what you want to do, or just calculate total exposure, this is unnecessary

In [ ]:
calculated_pdb_paths = calc_exposure(preprocessed_pdb_path, assignment=create_3_vectors(preprocessed_pdb_path, 'BDD', 'residue_name')) # By default, calculates with d^-2 and gives score and inverse score (better for ChimeraX visualisation)

for pdbpath in calculated_pdb_paths[-2:]
    average_score_per_residue(pdbpath) # Just averaging the first calculated

#### Getting local resolution by atom in ChimeraX
measure mapvalues #3 atoms #1 attribute locres

save 'C:\Users\chem-lady6261\OneDrive - Nexus365\Code\VS\pdbs\XXXX.defattr' attrName locres models #1

In [ ]:
plot_score_v_localres_byatom('pdbs/out/amtb_md_tot_2_avgbyres.pdb', 'pdbs/defattrs/J79_avgbyres.defattr', backboneonly=False)